In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
   ━━━━━━━━

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
!git clone https://github.com/QingruZhang/AdaLoRA.git
!pip install ./AdaLoRA/loralib

Cloning into 'AdaLoRA'...
remote: Enumerating objects: 1603, done.
remote: Counting objects: 100% (1603/1603), done.
remote: Compressing objects: 100% (781/781), done.
remote: Total 1603 (delta 818), reused 1601 (delta 816), pack-reused 0 (from 0)
Receiving objects: 100% (1603/1603), 4.20 MiB | 21.73 MiB/s, done.
Resolving deltas: 100% (818/818), done.
Processing ./AdaLoRA/loralib
  Preparing metadata (setup.py) ... done
  Created wheel for loralib: filename=loralib-0.1.0-py3-none-any.whl size=13657 sha256=692fb8c1abaab43b301bf300487217e8182268b6ad2a8aa1a93eb95b1264ff9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-vuftkldo/wheels/21/55/60/9a2650d6e6f04c298a199d836cbc5df341129aab52666fc7fe
Successfully built loralib


In [4]:
import warnings
warnings.filterwarnings("ignore")
import random
import numpy as np
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
)
import evaluate
from peft import AdaLoraConfig, get_peft_model, TaskType

2025-12-11 05:59:24.862695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765432765.063811      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765432765.119859      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [5]:
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
OUTPUT_DIR = "./boolq_lora_tinyllama_adaptive"
BATCH_SIZE = 16         
EVAL_BATCH = 32
NUM_EPOCHS = 3
LR = 1e-4
SEED = 42
MAX_LENGTH = 256
LORA_R = 8
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
TARGET_MODULES =["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
ADAPTIVE_TOTAL_RANK = 672
ADAPTIVE_TAU = 1.0

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

In [7]:
!pip install -U fsspec aiohttp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.5.0
    Uninstalling fsspec-2024.5.0:
      Successfully uninstalled fsspec-2024.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 2.20.0 requires fsspec[http]<=2024.5.0,>=2023.1.0, but you have fsspec 2025.12.0 which is incompatible.
s3fs 2025.3.0 requires fsspec==2025.3.0.*, but you have fsspec 2025.12.0 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.4 which is incompatible.
gc

In [8]:
from datasets import load_dataset
import time

def load_boolq_with_retry(max_retries=None, wait=2):
    attempt = 0

    while True:
        try:
            ds = load_dataset("boolq")
            print(f"✓ Dataset 'boolq' loaded successfully after {attempt} attempts.")
            return ds
        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt) 
            print(f"[Attempt {attempt}] Failed to load 'boolq': {e}")
            print(f"Retrying in {wait_time} seconds...\n")
            if max_retries is not None and attempt >= max_retries:
                print("✗ Maximum retries reached. Raising error.")
                raise e

            time.sleep(wait_time)
dataset = load_boolq_with_retry(max_retries=10)

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

✓ Dataset 'boolq' loaded successfully after 0 attempts.


In [9]:
train_ds = dataset["train"]
val_ds = dataset["validation"]
print("Train size:", len(train_ds))
print("Validation (test) size:", len(val_ds))

Train size: 9427
Validation (test) size: 3270


In [10]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [11]:
def clean_text(s):
    if s is None:
        return ""
    return " ".join(str(s).strip().split())


def preprocess(batch):
    questions = [clean_text(q) for q in batch["question"]]
    passages  = [clean_text(p) for p in batch["passage"]]
    reformatted_questions = [
        f"Question: {q}"
        for q in questions
    ]

    reformatted_passages = [
        f"Passage: {p}"
        for p in passages
    ]
    toks = tokenizer(
        reformatted_questions,
        reformatted_passages,
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
    )
    toks["labels"] = [1 if ans else 0 for ans in batch["answer"]]

    return toks


print("Preprocessing datasets...")
train_ds = train_ds.map(preprocess, batched=True)
val_ds = val_ds.map(preprocess, batched=True)

Preprocessing datasets...


Map:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
cols = ["input_ids", "attention_mask", "labels"]
train_ds.set_format(type="torch", columns=cols)
val_ds.set_format(type="torch", columns=cols)
train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(val_ds, batch_size=EVAL_BATCH, shuffle=False, collate_fn=data_collator)

In [13]:
print("Model started Loading")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True
)

Model started Loading


`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
print("Model Loaded")

Model Loaded


In [15]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

In [ ]:
from peft import AdaLoraConfig, TaskType
total_steps = 885  
adalora_config = AdaLoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    target_modules=TARGET_MODULES,
    init_r=12,
    target_r=LORA_R,
    tinit=int(0.1 * total_steps),
    tfinal=int(0.8 * total_steps),
    deltaT=max(1, int(0.04 * total_steps)),
    beta1=0.85,
    beta2=0.85,
    orth_reg_weight=1e-6,
    total_step=total_steps,
    init_lora_weights=True,
)

model = get_peft_model(model, adalora_config)

In [17]:
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=preds, references=labels)
    return {"accuracy": acc["accuracy"]}


In [18]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=200,
    learning_rate=LR,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    seed=SEED,                      
    gradient_accumulation_steps=2,      
    fp16=False,                             
    bf16=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\nStarting training on BoolQ...")
trainer.train()


Starting training on BoolQ...


Epoch,Training Loss,Validation Loss,Accuracy
1,1.375900,0.698198,0.631804
2,1.111000,0.654269,0.643731
3,1.093900,0.651132,0.660856


TrainOutput(global_step=3537, training_loss=1.2572927434531382, metrics={'train_runtime': 20801.9835, 'train_samples_per_second': 1.36, 'train_steps_per_second': 0.17, 'total_flos': 4.236638824606003e+16, 'train_loss': 1.2572927434531382, 'epoch': 3.0})

In [19]:
print("\nEvaluating final model...")
eval_result = trainer.evaluate(eval_dataset=val_ds)
test_accuracy = eval_result.get("eval_accuracy") or eval_result.get("accuracy")

print("\n=== Final testing (validation) results ===")
for k, v in eval_result.items():
    print(f"{k}: {v}")

model.save_pretrained(OUTPUT_DIR)
print(f"\nLoRA adapter + model saved to {OUTPUT_DIR}")


Evaluating final model...



=== Final testing (validation) results ===
eval_loss: 0.6511316299438477
eval_accuracy: 0.6608562691131499
eval_runtime: 1048.3891
eval_samples_per_second: 3.119
eval_steps_per_second: 1.56
epoch: 3.0

LoRA adapter + model saved to ./boolq_lora_tinyllama_adaptive
